![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [32]:

# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

import streamlit as st


In [11]:

# Load the dataset
data = pd.read_csv("cc_approvals.data", header=None, na_values="?") 
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [12]:
data.shape

(690, 14)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    float64
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(3), int64(2), object(9)
memory usage: 75.6+ KB


In [14]:
data.nunique()

0       2
1     349
2     215
3       3
4       3
5      14
6       9
7     132
8       2
9       2
10     23
11      3
12    240
13      2
dtype: int64

### Understanding the Dataset

The dataset has 690 rows and 14 columns. Here's a summary of the data:

- **Data Types**: There are various data types including `float64`, `int64`, and `object`.
- **Missing Values**: We will need to check for and handle any missing or erroneous values.
- **Target Variable**: The last column (index 13) is the target variable indicating whether the credit card application was approved (`+`) or not (`-`).

### Column Overview

The columns are not named, but for better understanding, let's assign temporary names based on the structure we see. The dataset appears to have both categorical and numerical features.

In [15]:
# Assign column names
column_names = [
    'Gender', 'Age', 'Debt', 'Marital_Status', 'Bank_Customer', 'Education_Level', 
    'Ethnicity', 'Years_Employed', 'Prior_Default', 'Employed', 'Credit_Score', 
    'Drivers_License', 'Citizen', 'Approval_Status'
]
data.columns = column_names

# Display unique values for each column to understand their nature
unique_values = {col: data[col].unique() for col in data.columns}

print(unique_values)

{'Gender': array(['b', 'a', nan], dtype=object), 'Age': array([30.83, 58.67, 24.5 , 27.83, 20.17, 32.08, 33.17, 22.92, 54.42,
       42.5 , 22.08, 29.92, 38.25, 48.08, 45.83, 36.67, 28.25, 23.25,
       21.83, 19.17, 25.  , 47.75, 27.42, 41.17, 15.83, 47.  , 56.58,
       57.42, 42.08, 29.25, 42.  , 49.5 , 36.75, 22.58, 27.25, 23.  ,
       27.75, 54.58, 34.17, 28.92, 29.67, 39.58, 56.42, 54.33, 41.  ,
       31.92, 41.5 , 23.92, 25.75, 26.  , 37.42, 34.92, 34.25, 23.33,
       23.17, 44.33, 35.17, 43.25, 56.75, 31.67, 23.42, 20.42, 26.67,
       36.  , 25.5 , 19.42, 32.33, 34.83, 38.58, 44.25, 44.83, 20.67,
       34.08, 21.67, 21.5 , 49.58, 27.67, 39.83,   nan, 37.17, 25.67,
       34.  , 49.  , 62.5 , 31.42, 52.33, 28.75, 28.58, 22.5 , 28.5 ,
       37.5 , 35.25, 18.67, 54.83, 40.92, 19.75, 29.17, 24.58, 33.75,
       25.42, 37.75, 52.5 , 57.83, 20.75, 39.92, 24.75, 44.17, 23.5 ,
       47.67, 22.75, 34.42, 28.42, 67.75, 47.42, 36.25, 32.67, 48.58,
       33.58, 18.83, 26.92, 31.25,

In [26]:
data.isnull().sum()

Gender             0
Age                0
Debt               0
Marital_Status     0
Bank_Customer      0
Education_Level    0
Ethnicity          0
Years_Employed     0
Prior_Default      0
Employed           0
Credit_Score       0
Drivers_License    0
Citizen            0
Approval_Status    0
dtype: int64

In [22]:
for col in ["Gender", "Marital_Status","Bank_Customer", "Education_Level", "Ethnicity"]:
    data[col].fillna(data[col].mode()[0], inplace=True)

In [24]:
data.dtypes

Gender              object
Age                float64
Debt               float64
Marital_Status      object
Bank_Customer       object
Education_Level     object
Ethnicity           object
Years_Employed     float64
Prior_Default       object
Employed            object
Credit_Score         int64
Drivers_License     object
Citizen              int64
Approval_Status     object
dtype: object

In [25]:
data["Age"].fillna(data["Age"].median(), inplace=True)

In [27]:
data.head()

,Gender,Age,Debt,Marital_Status,Bank_Customer,Education_Level,Ethnicity,Years_Employed,Prior_Default,Employed,Credit_Score,Drivers_License,Citizen,Approval_Status
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [31]:
# row duplicates
data.duplicated().sum()

0

# Encoding

In [35]:
data.columns

Index(['Gender', 'Age', 'Debt', 'Marital_Status', 'Bank_Customer',
       'Education_Level', 'Ethnicity', 'Years_Employed', 'Prior_Default',
       'Employed', 'Credit_Score', 'Drivers_License', 'Citizen',
       'Approval_Status'],
      dtype='object')